In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Flatten , Convolution2D, MaxPooling2D , Lambda, Conv2D, Activation,Concatenate, Input, BatchNormalization
from tensorflow.keras.optimizers import Adam , SGD , Adagrad
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers , initializers, activations
import tensorflow.keras.backend as K
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
import tensorflow_probability as tfp

import numpy as np
import matplotlib.pyplot as plt
import corner
import os 
import sys
import time
import importlib
import logging
from tqdm import tqdm

importlib.reload(logging)
logging.basicConfig(level = logging.INFO)

# limit GPU memory
gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
except RuntimeError as e:
# Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


Ref: https://agustinus.kristia.de/techblog/2016/12/17/conditional-vae/   
Ref: https://keras.io/guides/customizing_what_happens_in_fit/   
Ref: https://keras.io/examples/generative/vae/   
Ref: https://github.com/hagabbar/VItamin/blob/c1ae6dfa27b8ab77193caacddd477fde0dece1c2/Models/VICI_inverse_model.py#L404   
Ref: https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/MultivariateNormalDiag    

In [2]:
def angle_transform(prediction_array: np.ndarray = 0)-> np.ndarray :
    tmp = np.arctan2(prediction_array[:,0],prediction_array[:,1])
    tmp = np.where(tmp >= 0 , tmp, 2*np.pi+tmp) 
    delta = tmp/np.pi*180     
    return delta

In [3]:
%%time
training_data = np.load("./nsi_data/sample_nsi_regression_1e7_v1.npz")
data_all = np.column_stack([training_data['ve_dune'][:,:36], training_data['vu_dune'][:,:36], training_data['vebar_dune'][:,:36], training_data['vubar_dune'][:,:36]])

scaler = StandardScaler()
scaler.fit(data_all)

target = np.column_stack([training_data["theta23"], training_data["delta"]/180*np.pi ])

x_train = data_all[:9000000]
y_train = target[:9000000]
y_train_angle = np.column_stack([y_train[:,0], np.sin(y_train[:,1]), np.cos(y_train[:,1])]) 

x_test = data_all[9000000:]
y_test = target[9000000:]
y_test_angle = np.column_stack([y_test[:,0], np.sin(y_test[:,1]), np.cos(y_test[:,1])]) 

x_train_poisson = np.random.poisson(x_train)/1000
x_test_poisson = np.random.poisson(x_test)/1000

CPU times: user 3min 6s, sys: 10.2 s, total: 3min 16s
Wall time: 3min 17s


In [4]:
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [5]:
latent_dim = 7

"""
Encoder 1 (parameter + spectrum)
"""
# parameter
encoder_parameter_inputs = layers.Input(shape=(3,),name = 'encoder_parameter_inputs')
x_parameter = layers.Dense(64, activation="relu", name = 'dense_parameter_1')(encoder_parameter_inputs)
x_parameter = layers.Dense(32, activation="relu", name = 'dense_parameter_2')(x_parameter)
x_parameter = layers.Dense(16, activation="relu", name = 'dense_parameter_3')(x_parameter)

# spectrum
encoder_spectrum_inputs = layers.Input(shape=(144),name = 'encoder_spectrum_inputs')
x_spectrum = layers.Dense(64, activation="relu", name = 'dense_spectrum_1')(encoder_spectrum_inputs)
x_spectrum = layers.Dense(32, activation="relu", name = 'dense_spectrum_2')(x_spectrum)
x_spectrum = layers.Dense(16, activation="relu", name = 'dense_spectrum_3')(x_spectrum)

# merged
mergedOut_Encoder_1 = Concatenate()([x_parameter,x_spectrum])

# sampling
z_mean = layers.Dense(latent_dim, name="z_mean")(mergedOut_Encoder_1)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(mergedOut_Encoder_1)
# z = Sampling(name = 'Sampling_encoder')([z_mean, z_log_var])

# build model
encoder_1 = keras.Model([encoder_parameter_inputs, encoder_spectrum_inputs], [z_mean, z_log_var], name="encoder_1")
encoder_1.summary()

Model: "encoder_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_parameter_inputs (Input [(None, 3)]          0                                            
__________________________________________________________________________________________________
encoder_spectrum_inputs (InputL [(None, 144)]        0                                            
__________________________________________________________________________________________________
dense_parameter_1 (Dense)       (None, 64)           256         encoder_parameter_inputs[0][0]   
__________________________________________________________________________________________________
dense_spectrum_1 (Dense)        (None, 64)           9280        encoder_spectrum_inputs[0][0]    
__________________________________________________________________________________________

In [12]:
"""
Encoder 2 (spectrum)
"""
# spectrum
encoder_spectrum_inputs = layers.Input(shape=(144,),name = 'encoder_spectrum_inputs')
x_spectrum = layers.Dense(64, activation="relu", name = 'dense_spectrum_1')(encoder_spectrum_inputs)
x_spectrum = layers.Dense(32, activation="relu", name = 'dense_spectrum_2')(x_spectrum)
x_spectrum = layers.Dense(16, activation="relu", name = 'dense_spectrum_3')(x_spectrum)

# sampling
guassian_number = 20
z_mean = layers.Dense(guassian_number*latent_dim, name="z_mean")(x_spectrum)
z_log_var = layers.Dense(guassian_number*latent_dim, name="z_log_var")(x_spectrum)
z_weight = layers.Dense(guassian_number, name="z_weight")(x_spectrum)
# z = Sampling(name = 'Sampling_encoder')([z_mean, z_log_var])

# build model
encoder_2 = keras.Model(encoder_spectrum_inputs, [z_mean, z_log_var, z_weight], name="encoder_2")
encoder_2.summary()

Model: "encoder_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_spectrum_inputs (InputL [(None, 144)]        0                                            
__________________________________________________________________________________________________
dense_spectrum_1 (Dense)        (None, 64)           9280        encoder_spectrum_inputs[0][0]    
__________________________________________________________________________________________________
dense_spectrum_2 (Dense)        (None, 32)           2080        dense_spectrum_1[0][0]           
__________________________________________________________________________________________________
dense_spectrum_3 (Dense)        (None, 16)           528         dense_spectrum_2[0][0]           
__________________________________________________________________________________________

In [13]:
"""
Decoder Model (latent + spectrum)
"""
latent_dim_2 = 3

decoder_latent_inputs = keras.Input(shape=(latent_dim,),name = 'decoder_latent_inputs')
x_latent = layers.Dense(64, activation="relu", name = 'dense_1')(decoder_latent_inputs)
x_latent = layers.Dense(32, activation="relu", name = 'dense_2')(x_latent)
x_latent = layers.Dense(16, activation="relu", name = 'dense_3')(x_latent)

# spectrum
decoder_spectrum_inputs = layers.Input(shape=(144,),name = 'decoder_spectrum_inputs')
x_spectrum = layers.Dense(64, activation="relu", name = 'dense_spectrum_1')(decoder_spectrum_inputs)
x_spectrum = layers.Dense(32, activation="relu", name = 'dense_spectrum_2')(x_spectrum)
x_spectrum = layers.Dense(16, activation="relu", name = 'dense_spectrum_3')(x_spectrum)

# merged
mergedOut_Decoder = Concatenate()([x_latent,x_spectrum])

z2_mean = layers.Dense(latent_dim_2, name="z_mean")(mergedOut_Decoder)
z2_log_var = layers.Dense(latent_dim_2, name="z_log_var")(mergedOut_Decoder)
# z2 = Sampling(name = 'Sampling_decoder')([z2_mean, z2_log_var])

decoder = keras.Model([decoder_latent_inputs, decoder_spectrum_inputs], [z2_mean, z2_log_var], name="decoder")
decoder.summary()

Model: "decoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_latent_inputs (InputLay [(None, 7)]          0                                            
__________________________________________________________________________________________________
decoder_spectrum_inputs (InputL [(None, 144)]        0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 64)           512         decoder_latent_inputs[0][0]      
__________________________________________________________________________________________________
dense_spectrum_1 (Dense)        (None, 64)           9280        decoder_spectrum_inputs[0][0]    
____________________________________________________________________________________________

In [14]:
#Ref: https://keras.io/guides/customizing_what_happens_in_fit/
#Ref: https://keras.io/examples/generative/vae/
#Ref: https://github.com/hagabbar/VItamin/blob/c1ae6dfa27b8ab77193caacddd477fde0dece1c2/Models/VICI_inverse_model.py#L404
class CVAE(keras.Model):
    def __init__(self, encoder1, encoder2, decoder, **kwargs):
        super(CVAE, self).__init__(**kwargs)
        self.encoder1 = encoder1  #(parameter + spectrum)
        self.encoder2 = encoder2  #(spectrum)
        self.decoder = decoder    #(latent + spectrum)
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
                ]

    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            SMALL_CONSTANT = 1e-12 # necessary to prevent the division by zero in many operations 
            
            # encoder1: 
            # input: [parameter, spectrum]
            # output: [mean, log_var]
            # mean.shape= (N, 15)  log_var.shape= (N, 15)   D(latent space) = 15
            z1_mean, z1_log_var = self.encoder1(x)  #(parameter + spectrum)
            
            # GET q(z|x,y)   #(parameter + spectrum)
            temp_var_q = SMALL_CONSTANT + tf.exp(z1_log_var)
            mvn_q = tfp.distributions.MultivariateNormalDiag(
                          loc=z1_mean,
                          scale_diag=tf.sqrt(temp_var_q))
            
            z1 = mvn_q.sample()
            
            # encoder2: 
            # input: spectrum
            # output: [mean, log_var, weight]
            # mean.shape= (N, 10*15)  log_var.shape= (N, 10*15) weight.shape= (N, 10)  
            # D(latent space)=15, D(mixture component)=10
            z2_mean, z2_log_var, z2_weight = self.encoder2(x[1])  #(spectrum)

            # mean->reshape= (N, 10, 15  log_var->reshape= (N, 10, 15) for create mixture model
            z2_mean = tf.reshape(z2_mean, (-1, guassian_number, 7))
            z2_log_var = tf.reshape(z2_log_var, (-1, guassian_number, 7))
            z2_weight = tf.reshape(z2_weight, (-1, guassian_number))

            # Get r1(z|y) mixture model   #(spectrum)
            temp_var_r1 = SMALL_CONSTANT + tf.exp(z2_log_var)
            bimix_gauss = tfp.distributions.MixtureSameFamily(
                          mixture_distribution=tfp.distributions.Categorical(logits=z2_weight),
                          components_distribution=tfp.distributions.MultivariateNormalDiag(
                          loc=z2_mean,
                          scale_diag=tf.sqrt(temp_var_r1)))
            
            z2 = bimix_gauss.sample()
            
            # decoder: 
            # input: [latent, spectrum]
            # output: [mean, log_var]
            # mean.shape= (N, 2)  log_var.shape= (N, 2) 
            reconstruction_mean, reconstruction_var = self.decoder([z1, x[1]])      #(latent + spectrum)
            
            # GET r2(x|z,y)    #(latent + spectrum)
            temp_var_r2 = SMALL_CONSTANT + tf.exp(reconstruction_var)
            reconstruction_parameter = tfp.distributions.MultivariateNormalDiag(
                                     loc=reconstruction_mean,
                                     scale_diag= tf.sqrt(temp_var_r2))
            
            r2 = reconstruction_parameter.sample()
        
            log_q_q = mvn_q.log_prob(z1)
            log_r1_q = bimix_gauss.log_prob(z1)               # evaluate the log prob of r1 at the q samples(z1)
            kl_loss = tf.reduce_mean(log_q_q - log_r1_q)      # average over batch
            
            reconstruction_parameter_loss = reconstruction_parameter.log_prob(y)
            reconstruction_loss = -1.0*tf.reduce_mean(reconstruction_parameter_loss)
            
            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [15]:
"""
Model Training
"""
cvae = CVAE(encoder_1,encoder_2,decoder)
cvae.compile(optimizer=keras.optimizers.Adam())
# vae.compile(optimizer=keras.optimizers.Adadelta())

check_list=[]
csv_logger = CSVLogger("./Training_loss/CVAE_1DCNN_training_log_v2.csv")
check_list.append(csv_logger)

In [16]:
%%time
cvae.fit(x = [y_train_angle, x_train_poisson],
         y = y_train_angle,
         batch_size=1000,
         epochs=300,
         verbose=1,
         shuffle=True)

Epoch 1/300
9000/9000 [==============================] - 62s 7ms/step - loss: 15.6708 - reconstruction_loss: 6.1583 - kl_loss: 0.1642
Epoch 2/300
9000/9000 [==============================] - 60s 7ms/step - loss: 3.5446 - reconstruction_loss: 3.2273 - kl_loss: 0.1473
Epoch 3/300
9000/9000 [==============================] - 61s 7ms/step - loss: 2.9369 - reconstruction_loss: 2.5458 - kl_loss: 0.2895
Epoch 4/300
9000/9000 [==============================] - 61s 7ms/step - loss: 2.5385 - reconstruction_loss: 1.1776 - kl_loss: 1.2481
Epoch 5/300
9000/9000 [==============================] - 60s 7ms/step - loss: 2.1844 - reconstruction_loss: 0.4891 - kl_loss: 1.6166
Epoch 6/300
9000/9000 [==============================] - 60s 7ms/step - loss: 1.9422 - reconstruction_loss: 0.2202 - kl_loss: 1.6864
Epoch 7/300
9000/9000 [==============================] - 59s 7ms/step - loss: 1.8196 - reconstruction_loss: 0.0713 - kl_loss: 1.7344
Epoch 8/300
9000/9000 [==============================] - 59s 7ms/ste

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



9000/9000 [==============================] - 59s 7ms/step - loss: 0.2742 - reconstruction_loss: -1.6383 - kl_loss: 1.9148
Epoch 148/300
9000/9000 [==============================] - 59s 7ms/step - loss: 0.2725 - reconstruction_loss: -1.6325 - kl_loss: 1.9142
Epoch 149/300
9000/9000 [==============================] - 59s 7ms/step - loss: 0.2691 - reconstruction_loss: -1.6365 - kl_loss: 1.9156
Epoch 150/300
 329/9000 [>.............................] - ETA: 57s - loss: 0.2223 - reconstruction_loss: -1.6915 - kl_loss: 1.9312

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



9000/9000 [==============================] - 59s 7ms/step - loss: 0.2576 - reconstruction_loss: -1.6425 - kl_loss: 1.9023
Epoch 158/300
9000/9000 [==============================] - 59s 7ms/step - loss: 0.2432 - reconstruction_loss: -1.6691 - kl_loss: 1.9169
Epoch 159/300
8969/9000 [============================>.] - ETA: 0s - loss: 0.2485 - reconstruction_loss: -1.6660 - kl_loss: 1.9193

...

9000/9000 [==============================] - 60s 7ms/step - loss: 0.3669 - reconstruction_loss: -1.4389 - kl_loss: 1.8471
Epoch 236/300
9000/9000 [==============================] - 59s 7ms/step - loss: 0.1765 - reconstruction_loss: -1.6914 - kl_loss: 1.8705
Epoch 237/300
4650/9000 [==============>...............] - ETA: 28s - loss: 0.1824 - reconstruction_loss: -1.6967 - kl_loss: 1.8796

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



9000/9000 [==============================] - 59s 7ms/step - loss: 0.1920 - reconstruction_loss: -1.6373 - kl_loss: 1.8364
Epoch 239/300
6177/9000 [===================>..........] - ETA: 18s - loss: 0.1806 - reconstruction_loss: -1.6439 - kl_loss: 1.8265

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



9000/9000 [==============================] - 60s 7ms/step - loss: 0.1602 - reconstruction_loss: -1.7088 - kl_loss: 1.8560
Epoch 285/300
9000/9000 [==============================] - 60s 7ms/step - loss: 0.1381 - reconstruction_loss: -1.7198 - kl_loss: 1.8629
Epoch 286/300
9000/9000 [==============================] - 58s 6ms/step - loss: 0.1462 - reconstruction_loss: -1.6914 - kl_loss: 1.8474
Epoch 287/300
9000/9000 [==============================] - 58s 6ms/step - loss: 0.1474 - reconstruction_loss: -1.7018 - kl_loss: 1.8453
Epoch 288/300
9000/9000 [==============================] - 58s 6ms/step - loss: 0.1452 - reconstruction_loss: -1.7099 - kl_loss: 1.8554
Epoch 289/300
9000/9000 [==============================] - 59s 7ms/step - loss: 0.1312 - reconstruction_loss: -1.6896 - kl_loss: 1.8333
Epoch 290/300
9000/9000 [==============================] - 59s 7ms/step - loss: 0.1534 - reconstruction_loss: -1.6963 - kl_loss: 1.8406
Epoch 291/300
9000/9000 [==============================] - 59s

In [17]:
path = './CVAE/DNN_angleTran_20g_{}'
index = 1
while os.path.isdir(path.format(index)):
    index += 1
path = path.format(index)
cvae.encoder1.save(path + "/encoder_1_test_v2.h5")
cvae.encoder2.save(path + "/encoder_2_test_v2.h5")
cvae.decoder.save(path + "/decoder_test_v2.h5")